## 2 Models with lags & differences

## #1

In [1]:
# pip install linearmodles
import pandas as pd

from linearmodels import PooledOLS # Fitting of Pooling model
from linearmodels import RandomEffects # Fitting of RE-model
from linearmodels import PanelOLS # Fitting og FE-model
from linearmodels import FirstDifferenceOLS # First difference model

from linearmodels.panel import compare # tables of regressions' outputs

In [2]:
df = pd.read_csv('Grunfeld.csv')
df.head()

,firm,year,inv,value,capital
0,1,1935,317.6,3078.5,2.8
1,1,1936,391.8,4661.7,52.6
2,1,1937,410.6,5387.1,156.9
3,1,1938,257.7,2792.2,209.2
4,1,1939,330.8,4313.2,203.4


In [3]:
panel_df = df.set_index(['firm', 'year'])
panel_df.head(20)

inv   value  capital
firm year                         
1    1935   317.6  3078.5      2.8
     1936   391.8  4661.7     52.6
     1937   410.6  5387.1    156.9
     1938   257.7  2792.2    209.2
     1939   330.8  4313.2    203.4
     1940   461.2  4643.9    207.2
     1941   512.0  4551.2    255.2
     1942   448.0  3244.1    303.7
     1943   499.6  4053.7    264.1
     1944   547.5  4379.3    201.6
     1945   561.2  4840.9    265.0
     1946   688.1  4900.9    402.2
     1947   568.9  3526.5    761.5
     1948   529.2  3254.7    922.4
     1949   555.1  3700.2   1020.1
     1950   642.9  3755.6   1099.0
     1951   755.9  4833.0   1207.7
     1952   891.2  4924.9   1430.5
     1953  1304.4  6241.7   1777.3
     1954  1486.7  5593.6   2226.3

In [4]:
panel_df['d_inv'] = panel_df.groupby(level=0)['inv'].diff()
panel_df['d_value'] = panel_df.groupby(level=0)['value'].diff()
panel_df['d_capital'] = panel_df.groupby(level=0)['capital'].diff()
panel_df.head(20)

inv   value  capital  d_inv  d_value  d_capital
firm year                                                    
1    1935   317.6  3078.5      2.8    NaN      NaN        NaN
     1936   391.8  4661.7     52.6   74.2   1583.2       49.8
     1937   410.6  5387.1    156.9   18.8    725.4      104.3
     1938   257.7  2792.2    209.2 -152.9  -2594.9       52.3
     1939   330.8  4313.2    203.4   73.1   1521.0       -5.8
     1940   461.2  4643.9    207.2  130.4    330.7        3.8
     1941   512.0  4551.2    255.2   50.8    -92.7       48.0
     1942   448.0  3244.1    303.7  -64.0  -1307.1       48.5
     1943   499.6  4053.7    264.1   51.6    809.6      -39.6
     1944   547.5  4379.3    201.6   47.9    325.6      -62.5
     1945   561.2  4840.9    265.0   13.7    461.6       63.4
     1946   688.1  4900.9    402.2  126.9     60.0      137.2
     1947   568.9  3526.5    761.5 -119.2  -1374.4      359.3
     1948   529.2  3254.7    922.4  -39.7   -271.8      160.9
     1949   555.1  3700.2   1020.1   25.9    445.5       97.7
     1950   642.9  3755.6   1099.0   87.8     55.4       78.9
     1951   755.9  4833.0   1207.7  113.0   1077.4      108.7
     1952   891.2  4924.9   1430.5  135.3     91.9      222.8
     1953  1304.4  6241.7   1777.3  413.2   1316.8      346.8
     1954  1486.7  5593.6   2226.3  182.3   -648.1      449.0

In [5]:
mod_pl = PooledOLS.from_formula(formula='d_inv~1+d_value+d_capital', data=panel_df)
mod_re = RandomEffects.from_formula(formula='d_inv~1+d_value+d_capital', data=panel_df)
mod_fe = PanelOLS.from_formula(formula='d_inv~1+d_value+d_capital+EntityEffects', data=panel_df)
mod_fd = FirstDifferenceOLS.from_formula(formula='d_inv~d_value+d_capital', data=panel_df)

res_pl = mod_pl.fit(cov_type='clustered', cluster_entity=True)
res_re = mod_re.fit(cov_type='clustered', cluster_entity=True)
res_fe = mod_fe.fit(cov_type='clustered', cluster_entity=True)
res_fd = mod_fd.fit(cov_type='clustered', cluster_entity=True)

compare({'Pool': res_pl, 'RE': res_re, 'FE': res_fe, 'FD':res_fd}, stars=True)

/opt/anaconda3/lib/python3.11/site-packages/linearmodels/panel/model.py:921: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)
/opt/anaconda3/lib/python3.11/site-packages/linearmodels/panel/model.py:2759: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)
/opt/anaconda3/lib/python3.11/site-packages/linearmodels/panel/model.py:1260: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)


,Pool,RE,FE,FD
Dep. Variable,d_inv,d_inv,d_inv,d_inv
Estimator,PooledOLS,RandomEffects,PanelOLS,FirstDifferenceOLS
No. Observations,190,190,190,180
Cov. Est.,Clustered,Clustered,Clustered,Clustered
R-squared,0.4089,0.4089,0.3645,0.4645
R-Squared (Within),0.3612,0.3612,0.3645,0.1046
R-Squared (Between),0.8467,0.8467,0.7727,-1.2935
R-Squared (Overall),0.4089,0.4089,0.4046,-0.0769
F-statistic,64.674,64.674,51.053,77.209
P-value (F-stat),0.0000,0.0000,0.0000,0.0000


In [6]:
import numpy as np
from scipy.stats import chi2

In [7]:
(res_fe.params-res_re.params).T@np.linalg.inv(res_fe.cov-res_re.cov)@(res_fe.params-res_re.params)

0.009503960311212027

In [8]:
chi2.isf(q=0.05, df=res_re.df_model-1)

5.991464547107983

The test statistic is lower than the critical point $\Rightarrow$ the RE model is the best

## #2

In [9]:
panel_df['lag_value'] = panel_df.groupby(level=0)['value'].shift()
panel_df['lag_capital'] = panel_df.groupby(level=0)['capital'].shift()

In [10]:
mod_pl = PooledOLS.from_formula(formula='inv~1+value+capital+lag_value+lag_capital', data=panel_df)
mod_re = RandomEffects.from_formula(formula='inv~1+value+capital+lag_value+lag_capital', data=panel_df)
mod_fe = PanelOLS.from_formula(formula='inv~1+value+capital+lag_value+lag_capital+EntityEffects', data=panel_df)
mod_fd = FirstDifferenceOLS.from_formula(formula='inv~value+capital+lag_value+lag_capital', data=panel_df)

res_pl = mod_pl.fit(cov_type='clustered', cluster_entity=True)
res_re = mod_re.fit(cov_type='clustered', cluster_entity=True)
res_fe = mod_fe.fit(cov_type='clustered', cluster_entity=True)
res_fd = mod_fd.fit(cov_type='clustered', cluster_entity=True)

compare({'Pool': res_pl, 'RE': res_re, 'FE': res_fe, 'FD':res_fd}, stars=True)

/opt/anaconda3/lib/python3.11/site-packages/linearmodels/panel/model.py:921: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)
/opt/anaconda3/lib/python3.11/site-packages/linearmodels/panel/model.py:2759: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)
/opt/anaconda3/lib/python3.11/site-packages/linearmodels/panel/model.py:1260: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)
/opt/anaconda3/lib/python3.11/site-packages/linearmodels/panel/model.py:2427: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)


,Pool,RE,FE,FD
Dep. Variable,inv,inv,inv,inv
Estimator,PooledOLS,RandomEffects,PanelOLS,FirstDifferenceOLS
No. Observations,190,190,190,180
Cov. Est.,Clustered,Clustered,Clustered,Clustered
R-squared,0.8266,0.8000,0.7985,0.4841
R-Squared (Within),0.7714,0.7982,0.7985,0.7578
R-Squared (Between),0.8429,0.8204,0.8109,0.8160
R-Squared (Overall),0.8266,0.8153,0.8081,0.8069
F-statistic,220.49,184.96,174.33,41.287
P-value (F-stat),0.0000,0.0000,0.0000,0.0000


In [11]:
 (res_fe.params-res_re.params).T@np.linalg.inv(res_fe.cov-res_re.cov)@(res_fe.params-res_re.params)

0.5970147748316306

In [12]:
chi2.isf(q=0.05, df=res_re.df_model-1)

9.487729036781158

The test statistic is lower than the critical point $\Rightarrow$ the RE model is the best

## #3 The exercise with the dataframe EmplUK.csv

In [13]:
df = pd.read_csv('EmplUK.csv')
df.head()

,firm,year,sector,emp,wage,capital,output
0,1,1977,7,5.041,13.1516,0.5894,95.707199
1,1,1978,7,5.600,12.3018,0.6318,97.356903
2,1,1979,7,5.015,12.8395,0.6771,99.608299
3,1,1980,7,4.715,13.8039,0.6171,100.550100
4,1,1981,7,4.093,14.2897,0.5076,99.558098


In [14]:
panel_df = df.set_index(['firm', 'year'])

In [15]:
panel_df['l_emp'] = np.log(panel_df['emp'])
panel_df['d_l_emp'] = panel_df.groupby(level=0)['l_emp'].diff()

panel_df['l_capital'] = np.log(panel_df['capital'])
panel_df['d_l_capital'] = panel_df.groupby(level=0)['l_capital'].diff()

panel_df['l_wage'] = np.log(panel_df['wage'])
panel_df['d_l_wage'] = panel_df.groupby(level=0)['l_wage'].diff()

panel_df['l_output'] = np.log(panel_df['output'])
panel_df['d_l_output'] = panel_df.groupby(level=0)['l_output'].diff()

panel_df.head(20)

sector        emp       wage    capital      output     l_emp  \
firm year                                                                  
1    1977       7   5.041000  13.151600   0.589400   95.707199  1.617604   
     1978       7   5.600000  12.301800   0.631800   97.356903  1.722767   
     1979       7   5.015000  12.839500   0.677100   99.608299  1.612433   
     1980       7   4.715000  13.803900   0.617100  100.550100  1.550749   
     1981       7   4.093000  14.289700   0.507600   99.558098  1.409278   
     1982       7   3.166000  14.868100   0.422900   98.615097  1.152469   
     1983       7   2.936000  13.778400   0.392000  100.030100  1.077048   
2    1977       7  71.319000  14.790900  16.936300   95.707199  4.267163   
     1978       7  70.642998  14.103600  17.242201   97.356903  4.257639   
     1979       7  70.917999  14.953400  17.541300   99.608299  4.261524   
     1980       7  72.030998  15.491000  17.657400  100.550100  4.277097   
     1981       7  73.689003  16.196899  16.713301   99.558098  4.299854   
     1982       7  72.418999  16.131399  16.246901   98.615097  4.282469   
     1983       7  68.517998  16.305099  17.369600  100.030100  4.227096   
3    1977       7  19.156000  22.691999   7.097500   95.707199  2.952616   
     1978       7  19.440001  20.693800   6.946900   97.356903  2.967333   
     1979       7  19.900000  21.204800   6.856500   99.608299  2.990720   
     1980       7  20.240000  22.197001   6.654700  100.550100  3.007661   
     1981       7  19.570000  24.871401   6.213600   99.558098  2.973998   
     1982       7  18.125000  24.844700   5.714600   98.615097  2.897292   

            d_l_emp  l_capital  d_l_capital    l_wage  d_l_wage  l_output  \
firm year                                                                   
1    1977       NaN  -0.528650          NaN  2.576543       NaN  4.561294   
     1978  0.105162  -0.459182     0.069468  2.509746 -0.066798  4.578384   
     1979 -0.110333  -0.389936     0.069246  2.552526  0.042781  4.601245   
     1980 -0.061684  -0.482724    -0.092788  2.624951  0.072425  4.610656   
     1981 -0.141471  -0.678062    -0.195337  2.659539  0.034588  4.600741   
     1982 -0.256809  -0.860620    -0.182558  2.699218  0.039679  4.591224   
     1983 -0.075421  -0.936493    -0.075874  2.623102 -0.076116  4.605471   
2    1977       NaN   2.829459          NaN  2.694012       NaN  4.561294   
     1978 -0.009524   2.847360     0.017901  2.646430 -0.047582  4.578384   
     1979  0.003885   2.864558     0.017198  2.704939  0.058509  4.601245   
     1980  0.015572   2.871155     0.006597  2.740259  0.035321  4.610656   
     1981  0.022757   2.816205    -0.054950  2.784820  0.044561  4.600741   
     1982 -0.017385   2.787902    -0.028303  2.780768 -0.004052  4.591224   
     1983 -0.055372   2.854722     0.066819  2.791478  0.010710  4.605471   
3    1977       NaN   1.959743          NaN  3.122012       NaN  4.561294   
     1978  0.014717   1.938296    -0.021447  3.029834 -0.092178  4.578384   
     1979  0.023387   1.925197    -0.013098  3.054228  0.024393  4.601245   
     1980  0.016941   1.895323    -0.029874  3.099957  0.045730  4.610656   
     1981 -0.033663   1.826740    -0.068583  3.213719  0.113761  4.600741   
     1982 -0.076706   1.743024    -0.083716  3.212644 -0.001074  4.591224   

           d_l_output  
firm year              
1    1977         NaN  
     1978    0.017090  
     1979    0.022862  
     1980    0.009411  
     1981   -0.009915  
     1982   -0.009517  
     1983    0.014247  
2    1977         NaN  
     1978    0.017090  
     1979    0.022862  
     1980    0.009411  
     1981   -0.009915  
     1982   -0.009517  
     1983    0.014247  
3    1977         NaN  
     1978    0.017090  
     1979    0.022862  
     1980    0.009411  
     1981   -0.009915  
     1982   -0.009517

In [16]:
mod_pl = PooledOLS.from_formula(formula='d_l_emp~1+d_l_capital+d_l_wage+d_l_output', data=panel_df)
mod_re = RandomEffects.from_formula(formula='d_l_emp~1+d_l_capital+d_l_wage+d_l_output', data=panel_df)
mod_fe = PanelOLS.from_formula(formula='d_l_emp~1+d_l_capital+d_l_wage+d_l_output+EntityEffects', data=panel_df)
mod_fd = FirstDifferenceOLS.from_formula(formula='d_l_emp~d_l_capital+d_l_wage+d_l_output', data=panel_df)

res_pl = mod_pl.fit(cov_type='clustered', cluster_entity=True)
res_re = mod_re.fit(cov_type='clustered', cluster_entity=True)
res_fe = mod_fe.fit(cov_type='clustered', cluster_entity=True)
res_fd = mod_fd.fit(cov_type='clustered', cluster_entity=True)

compare({'Pool': res_pl, 'RE': res_re, 'FE': res_fe, 'FD':res_fd}, stars=True)

/opt/anaconda3/lib/python3.11/site-packages/linearmodels/panel/model.py:921: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)
/opt/anaconda3/lib/python3.11/site-packages/linearmodels/panel/model.py:2759: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)
/opt/anaconda3/lib/python3.11/site-packages/linearmodels/panel/model.py:1260: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)


,Pool,RE,FE,FD
Dep. Variable,d_l_emp,d_l_emp,d_l_emp,d_l_emp
Estimator,PooledOLS,RandomEffects,PanelOLS,FirstDifferenceOLS
No. Observations,891,891,891,668
Cov. Est.,Clustered,Clustered,Clustered,Clustered
R-squared,0.3830,0.3830,0.3693,0.3020
R-Squared (Within),0.3644,0.3644,0.3693,0.3485
R-Squared (Between),0.4687,0.4687,0.4190,0.4871
R-Squared (Overall),0.3830,0.3830,0.3780,0.3834
F-statistic,183.51,183.51,146.03,95.908
P-value (F-stat),0.0000,0.0000,0.0000,0.0000


In [17]:
 (res_fe.params-res_re.params).T@np.linalg.inv(res_fe.cov-res_re.cov)@(res_fe.params-res_re.params)

7.283070546413963

In [18]:
chi2.isf(q=0.05, df=res_re.df_model-1)

7.814727903251178

The test statistic is lower than the critical point $\Rightarrow$ the RE model is the best

In [24]:
panel_df['lag_l_wage'] = panel_df.groupby(level=0)['l_wage'].shift()
panel_df['lag_l_capital'] = panel_df.groupby(level=0)['l_capital'].shift()
panel_df['lag_l_output'] = panel_df.groupby(level=0)['l_output'].shift()

In [26]:
mod_pl = PooledOLS.from_formula(formula='l_emp~1+d_l_capital+lag_l_capital+d_l_wage+lag_l_wage+d_l_output+lag_l_output', data=panel_df)
mod_re = RandomEffects.from_formula(formula='l_emp~1+d_l_capital+lag_l_capital+d_l_wage+lag_l_wage+d_l_output+lag_l_output', data=panel_df)
mod_fe = PanelOLS.from_formula(formula='l_emp~1+d_l_capital+lag_l_capital+d_l_wage+lag_l_wage+d_l_output+lag_l_output+EntityEffects', data=panel_df)
mod_fd = FirstDifferenceOLS.from_formula(formula='l_emp~d_l_capital+lag_l_capital+d_l_wage+lag_l_wage+d_l_output+lag_l_output', data=panel_df)

res_pl = mod_pl.fit(cov_type='clustered', cluster_entity=True)
res_re = mod_re.fit(cov_type='clustered', cluster_entity=True)
res_fe = mod_fe.fit(cov_type='clustered', cluster_entity=True)
res_fd = mod_fd.fit(cov_type='clustered', cluster_entity=True)

compare({'Pool': res_pl, 'RE': res_re, 'FE': res_fe, 'FD':res_fd}, stars=True)

/opt/anaconda3/lib/python3.11/site-packages/linearmodels/panel/model.py:921: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)
/opt/anaconda3/lib/python3.11/site-packages/linearmodels/panel/model.py:2759: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)
/opt/anaconda3/lib/python3.11/site-packages/linearmodels/panel/model.py:1260: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)
/opt/anaconda3/lib/python3.11/site-packages/linearmodels/panel/model.py:2427: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)


,Pool,RE,FE,FD
Dep. Variable,l_emp,l_emp,l_emp,l_emp
Estimator,PooledOLS,RandomEffects,PanelOLS,FirstDifferenceOLS
No. Observations,891,891,891,668
Cov. Est.,Clustered,Clustered,Clustered,Clustered
R-squared,0.8406,0.7036,0.6459,0.5416
R-Squared (Within),0.4528,0.6369,0.6459,0.6394
R-Squared (Between),0.8545,0.8292,0.7768,0.4695
R-Squared (Overall),0.8406,0.8195,0.7696,0.4681
F-statistic,776.98,349.73,226.44,130.35
P-value (F-stat),0.0000,0.0000,0.0000,0.0000


In [27]:
 (res_fe.params-res_re.params).T@np.linalg.inv(res_fe.cov-res_re.cov)@(res_fe.params-res_re.params)

-56.28316825780087

In [28]:
chi2.isf(q=0.05, df=res_re.df_model-1)

12.59158724374398

The test statistic is lower than the critical point $\Rightarrow$ the RE model is the best